In [7]:
%pip install datasets
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("hendzh/PromptShield")

  Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
  Using cached pyarrow-20.0.0-cp310-cp310-win_amd64.whl (25.8 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\Aventus3\venv\Scripts\python.exe -m pip install --upgrade pip' command.
d:\Aventus3\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
%pip install protobuf
from transformers import AutoTokenizer


model_name = "jackaduma/SecBERT"

tokenizer = AutoTokenizer.from_pretrained(model_name)
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

You should consider upgrading via the 'd:\Aventus3\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at jackaduma/SecBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'label'],
        num_rows: 18909
    })
    validation: Dataset({
        features: ['prompt', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['prompt', 'label'],
        num_rows: 23516
    })
})


In [26]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "jackaduma/SecBERT"
MAX_LEN = 256
BATCH_SIZE = 8
EPOCHS = 3
LR = 2e-5


In [27]:
# Load from Hugging Face
dataset = load_dataset("hendzh/PromptShield")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(example):
    return tokenizer(example["prompt"], padding="max_length", truncation=True, max_length=MAX_LEN)

tokenized = dataset.map(tokenize_fn, batched=True)

Map: 100%|██████████| 23516/23516 [00:04<00:00, 4856.16 examples/s]


In [32]:
class PromptShieldDataset(Dataset):
    def __init__(self, hf_dataset):
        self.dataset = hf_dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        return {
            "input_ids": torch.tensor(item["input_ids"]),
            "attention_mask": torch.tensor(item["attention_mask"]),
            "label": torch.tensor(item["label"])
        }

train_dataset = PromptShieldDataset(tokenized["train"])
val_dataset = PromptShieldDataset(tokenized["validation"])
test_dataset = PromptShieldDataset(tokenized["test"])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [33]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at jackaduma/SecBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Training loss = {total_loss / len(train_loader):.4f}")

Epoch 1:   5%|▌         | 120/2364 [32:41<10:11:18, 16.35s/it]


KeyboardInterrupt: 